In [3]:
%load_ext autoreload
%autoreload

import os
import sys

import numpy as np 
import pandas as pd
from pathlib import Path

try:
    from ClearMap.Analysis.Graphs.GraphGt import load as load_graph
except:
    sys.path.append(f"/home/{os.getlogin()}/programs/ClearMap2")
    from ClearMap.Analysis.Graphs.GraphGt import load as load_graph
    import ClearMap.Settings as settings

try:
    from src.graph_visualization import graph_vis
except:
    sys.path.append(f"/home/{os.getlogin()}/programs/stats3d")
    from src.graph_visualization import graph_vis

ontology_df = pd.read_json(Path(settings.atlas_folder) / 'ABA_annotation_last.jsonl', lines=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Vascular graph loading

In [ ]:
print('Loading graph')
graph_path = ''
graph = load_graph(graph_path)

In [ ]:
print('Available edges properties: ')
print(list(graph.edge_properties))
print('Available vertex properties: ')
print(list(graph.vertex_properties))

### Recommended: subslice the graph

In [ ]:
gs = graph.sub_slice((slice(0, 4000), slice(0, 4000), slice(2100, 2200)))

### Plot colored by vessel type (veins / arteries)

In [18]:
# General color: grey 
n_edges = len(gs.edge_property('length'))
edge_color = np.array([[100, 100, 100, 100] for _ in range(n_edges)])

# Arteries color: red
edge_color[gs.edge_property('artery')] = [255, 0, 0, 255]

# Veins color: blue 
edge_color[gs.edge_property('vein')] = [0, 0, 255, 255]

scale = np.array([1.625, 1.625, 2])
graph_vis.plot_pyvista_plot(gs, edge_color, scale=scale)

Widget(value='<iframe src="http://localhost:41359/index.html?ui=P_0x7f378fe8ab90_4&reconnect=auto" class="pyvi…

### Highlight specific region

In [12]:
# id of the regions to highlight: 
id_list = [515,523,272,133]

In [17]:
# General color: grey 
n_edges = len(gs.edge_property('length'))
edge_color = np.array([[100, 100, 100, 255] for _ in range(n_edges)])

# Region of interest color: red
vertex_filter = np.isin(gs.vertex_property('annotation'), id_list)
edge_filter = graph_vis.vertex_filter_to_edge_filter(gs, vertex_filter)
edge_color[edge_filter] = [255, 0, 0, 255]

scale = np.array([1.625, 1.625, 2])
graph_vis.plot_pyvista_plot(gs, edge_color, scale=scale)

Widget(value='<iframe src="http://localhost:41359/index.html?ui=P_0x7f37c2ef0d90_3&reconnect=auto" class="pyvi…

### Plot colored by atlas annotation

In [ ]:
# Create a mapping from annotation id to RGB color
# Convert hex color to RGB tuple
def hex_to_rgb(hex_color):
    """Convert hex color string (e.g., 'B0FFB8') to RGB tuple (0-255)"""
    hex_color = hex_color.strip()
    return (
        int(hex_color[0:2], 16),
        int(hex_color[2:4], 16),
        int(hex_color[4:6], 16)
    )

color_map = ontology_df.set_index('id')['color_hex_triplet'].apply(hex_to_rgb).to_dict()

# Get unique annotation IDs and create colors for each
vertex_annotations = gs.vertex_property('annotation')
unique_annotations = np.unique(vertex_annotations)
n_edges = len(gs.edge_property('length'))
edge_color = np.zeros((n_edges, 4), dtype=np.uint8)

# For each annotation, create a vertex filter and transfer to edges
for annotation_id in unique_annotations:
    # Create vertex filter for this annotation
    vertex_filter = vertex_annotations == annotation_id
    
    # Transfer to edge filter
    edge_filter = graph_vis.vertex_filter_to_edge_filter(gs, vertex_filter)
    
    # Get color for this annotation
    if annotation_id in color_map:
        r, g, b = color_map[annotation_id]
        edge_color[edge_filter] = [r, g, b, 255]  # Full opacity
    else:
        # Default grey color for unmapped annotations
        edge_color[edge_filter] = [100, 100, 100, 255]

# Plot the graph
scale = np.array([1.625, 1.625, 2])
graph_vis.plot_pyvista_plot(gs, edge_color, scale=scale)

Widget(value='<iframe src="http://localhost:41359/index.html?ui=P_0x7f3814398b80_2&reconnect=auto" class="pyvi…